In [1]:
!which python

/home/user/jfayzullaev/stellar-clustering/.venv-vis/bin/python


In [6]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI


In [3]:
RES_FILE = "../louvain_result_res0.5.csv"


RAW_LABELS_PATH  = os.path.expanduser("~/stellar-clustering/network/labled-data/labels/labels_map.csv")
NORM_LABELS_PATH = os.path.expanduser("~/stellar-clustering/network/labled-data/labels/label-normalization/labels_entities_normalized.csv")


In [4]:
N_SPLITS = 5
RANDOM_STATE = 42

In [5]:
def overall_purity(df_comm_name: pd.DataFrame) -> float:

    if df_comm_name.empty:
        return np.nan
    counts = df_comm_name.groupby(['community', 'name']).size().reset_index(name='cnt')
    totals = counts.groupby('community')['cnt'].sum()
    max_per_comm = counts.groupby('community')['cnt'].max()
    return float(max_per_comm.sum() / totals.sum())


def load_communities_fixed_resolution(res_file: str) -> pd.DataFrame:

    if not os.path.exists(res_file):
        raise FileNotFoundError(f"Missing Louvain file: {res_file}")
    df = pd.read_csv(res_file)
    if 'account_id' not in df.columns and 'node' in df.columns:
        df = df.rename(columns={'node': 'account_id'})
    df = df[['account_id', 'community']].dropna().drop_duplicates()

    try:
        df['account_id'] = df['account_id'].astype(int)
    except Exception:
        df['account_id'] = df['account_id'].astype(str)
    return df


In [ ]:
# 5 fold
def evaluate_fixed_resolution_cv(
    labels_path: str,
    comm_df: pd.DataFrame,
    label_col: str = "name",
    n_splits: int = 5,
    random_state: int = 42
):

    if not os.path.exists(labels_path):
        raise FileNotFoundError(f"Labels file not found: {labels_path}")

    labels = (pd.read_csv(labels_path)
                .dropna(subset=['account_id', label_col])
                .drop_duplicates(subset=['account_id'])
                .rename(columns={label_col: 'name'}))


    try:
        labels['account_id'] = labels['account_id'].astype(int)
        comm_cast = comm_df.copy()
        comm_cast['account_id'] = comm_cast['account_id'].astype(int)
    except Exception:
        labels['account_id'] = labels['account_id'].astype(str)
        comm_cast = comm_df.copy()
        comm_cast['account_id'] = comm_cast['account_id'].astype(str)

    # make joins only accounts that in this louvain result
    joined = labels.merge(comm_cast, on='account_id', how='inner')

    n_labeled = len(labels)
    n_joined  = len(joined)
    coverage  = (n_joined / n_labeled) if n_labeled else 0.0
    if n_joined == 0:
        raise ValueError("No labeled accounts found in this Louvain file after join.")


    le = LabelEncoder()
    y_all = le.fit_transform(joined['name'].values)
    X_ids = joined['account_id'].values

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_ids, y_all), start=1):
        df_tr = joined.iloc[tr_idx].copy()
        df_te = joined.iloc[te_idx].copy()

        y_true_tr = le.transform(df_tr['name'])
        y_pred_tr = df_tr['community'].values
        nmi_tr = NMI(y_true_tr, y_pred_tr)
        ari_tr = ARI(y_true_tr, y_pred_tr)
        purity_tr = overall_purity(df_tr[['community', 'name']])


        y_true_te = le.transform(df_te['name'])
        y_pred_te = df_te['community'].values
        nmi_te = NMI(y_true_te, y_pred_te)
        ari_te = ARI(y_true_te, y_pred_te)
        purity_te = overall_purity(df_te[['community', 'name']])

        rows.append({
            'fold': fold,
            'n_train': len(df_tr),
            'n_test': len(df_te),
            'train_frac': len(df_tr) / len(joined),

            'NMI_train': nmi_tr,
            'ARI_train': ari_tr,
            'Purity_train': purity_tr,

            'NMI_test': nmi_te,
            'ARI_test': ari_te,
            'Purity_test': purity_te,
        })

    per_fold_df = pd.DataFrame(rows)

    averages = {
        'Avg_NMI_train': float(np.nanmean(per_fold_df['NMI_train'].values)),
        'Avg_ARI_train': float(np.nanmean(per_fold_df['ARI_train'].values)),
        'Avg_Purity_train': float(np.nanmean(per_fold_df['Purity_train'].values)),

        'Avg_NMI_test': float(np.nanmean(per_fold_df['NMI_test'].values)),
        'Avg_ARI_test': float(np.nanmean(per_fold_df['ARI_test'].values)),
        'Avg_Purity_test': float(np.nanmean(per_fold_df['Purity_test'].values)),

        'Avg_train_frac': float(np.nanmean(per_fold_df['train_frac'].values)),
    }

    coverage_info = {
        'n_labeled': int(n_labeled),
        'n_joined': int(n_joined),
        'coverage': float(coverage),
    }

    return per_fold_df, averages, coverage_info


In [8]:
comm_df = load_communities_fixed_resolution(RES_FILE)
print(f"Loaded communities from: {RES_FILE}")
print(f"Rows: {len(comm_df):,}  |  Unique accounts: {comm_df['account_id'].nunique():,}")


Loaded communities from: ../louvain_result_res0.5.csv
Rows: 206,451  |  Unique accounts: 206,451


### RAW LABELS

In [ ]:
raw_per_fold, raw_avg, raw_cov = evaluate_fixed_resolution_cv(
    labels_path=RAW_LABELS_PATH,
    comm_df=comm_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

print("RAW labels: per-fold metrics (TEST)")
display(raw_per_fold)

print("\nRAW labels: averages (TEST)")
for k, v in raw_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {raw_cov['coverage']:.2%}  ({raw_cov['n_joined']}/{raw_cov['n_labeled']})")


raw_per_fold.to_csv("cv_results_raw_per_fold.csv", index=False)
pd.DataFrame([{
    **raw_avg,
    **raw_cov
}]).to_csv("cv_results_raw_summary.csv", index=False)
print("\nSaved: cv_results_raw_per_fold.csv, cv_results_raw_summary.csv")


RAW labels: per-fold metrics (TEST)


/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.317248,0.063609,0.277603,0.401406,0.096449,0.300000
1,2,317,80,0.798489,0.322399,0.067655,0.271293,0.361803,0.065837,0.300000
2,3,318,79,0.801008,0.306918,0.067883,0.273585,0.434803,0.074627,0.316456
3,4,318,79,0.801008,0.315857,0.069911,0.283019,0.383001,0.047258,0.291139
4,5,318,79,0.801008,0.314330,0.076883,0.289308,0.393884,0.032973,0.253165



RAW labels: averages (TEST)
Avg_NMI_train: 0.315350
Avg_ARI_train: 0.069188
Avg_Purity_train: 0.278962
Avg_NMI_test: 0.394979
Avg_ARI_test: 0.063429
Avg_Purity_test: 0.292152
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)

Saved: cv_results_raw_per_fold.csv, cv_results_raw_summary.csv


### NORMALIZED LABELS

In [ ]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_resolution_cv(
    labels_path=NORM_LABELS_PATH,
    comm_df=comm_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

print("NORMALIZED labels: per-fold metrics (TEST)")
display(norm_per_fold)

print("\nNORMALIZED labels: averages (TEST)")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")


norm_per_fold.to_csv("cv_results_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov
}]).to_csv("cv_results_norm_summary.csv", index=False)
print("\nSaved: cv_results_norm_per_fold.csv, cv_results_norm_summary.csv")


NORMALIZED labels: per-fold metrics (TEST)


/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.247654,0.014278,0.731861,0.290781,0.020593,0.737500
1,2,317,80,0.798489,0.250429,0.017384,0.731861,0.254223,0.007753,0.725000
2,3,318,79,0.801008,0.243486,0.012170,0.726415,0.291017,0.020324,0.746835
3,4,318,79,0.801008,0.251907,0.019315,0.729560,0.254939,0.002315,0.734177
4,5,318,79,0.801008,0.247686,0.021186,0.732704,0.250241,-0.005842,0.721519



NORMALIZED labels: averages (TEST)
Avg_NMI_train: 0.248233
Avg_ARI_train: 0.016867
Avg_Purity_train: 0.730480
Avg_NMI_test: 0.268240
Avg_ARI_test: 0.009028
Avg_Purity_test: 0.733006
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)

Saved: cv_results_norm_per_fold.csv, cv_results_norm_summary.csv


In [ ]:
compare = pd.DataFrame([
    {
        "labels": "RAW",
        "Avg_NMI_test": raw_avg["Avg_NMI_test"],
        "Avg_ARI_test": raw_avg["Avg_ARI_test"],
        "Avg_Purity_test": raw_avg["Avg_Purity_test"],
        "coverage": raw_cov["coverage"],
        "n_labeled": raw_cov["n_labeled"],
        "n_joined": raw_cov["n_joined"],
    },
    {
        "labels": "NORMALIZED",
        "Avg_NMI_test": norm_avg["Avg_NMI_test"],
        "Avg_ARI_test": norm_avg["Avg_ARI_test"],
        "Avg_Purity_test": norm_avg["Avg_Purity_test"],
        "coverage": norm_cov["coverage"],
        "n_labeled": norm_cov["n_labeled"],
        "n_joined": norm_cov["n_joined"],
    },
])

compare["Diff_NMI"] = compare["Avg_NMI_test"].diff().fillna(np.nan)
compare["Diff_ARI"] = compare["Avg_ARI_test"].diff().fillna(np.nan)
compare["Diff_Purity"] = compare["Avg_Purity_test"].diff().fillna(np.nan)

print("RAW vs NORMALIZED (averages, TEST)")
display(compare)

compare.to_csv("cv_results_compare_raw_vs_normalized.csv", index=False)
print("\nSaved: cv_results_compare_raw_vs_normalized.csv")


=== RAW vs NORMALIZED (averages, TEST) ===


,labels,Avg_NMI_test,Avg_ARI_test,Avg_Purity_test,coverage,n_labeled,n_joined,Diff_NMI,Diff_ARI,Diff_Purity
0,RAW,0.394979,0.063429,0.292152,0.049,8102,397,NaN,NaN,NaN
1,NORMALIZED,0.268240,0.009028,0.733006,0.049,8102,397,-0.126739,-0.0544,0.440854



Saved: cv_results_compare_raw_vs_normalized.csv
